# ST446 Distributed Computing for Big Data
## Homework PART 1
---


## P1: Querying the YAGO semantic knowledge base

YAGO is a semantic knowledge base, derived from Wikipedia, WordNet and GeoNames. YAGO contains knowledge about more than 10 million entities (like persons, organizations and cities) and contains more than 120 million facts about these entities. You may find more about YAGO [here](https://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/yago/#c10444).

*You may use GCP or your own computer. Please document your steps. We highly recommend using GCP, as the data sets used are about 20 GB in total.*

In this homework assignment, you are asked to use parts of the YAGO dataset to demonstrate your knowledge about Spark graphframes and motif queries. In particular, you are asked to **_use motif queries_** to find out answers to the following queries stated in English:

**A (max points 0)**. _Which city was Albert Einstein born in?_ 

**B (max points 5)**. _Politicians who are also scientists_ (sorted alphabetically by name of person)

**C (max points 5)**. _Companies whose founders were born in London_ (sorted alphabetically by name of founder)

**D (max points 5)**. _Writers who have won a Nobel Prize (in any discipline)_ (sorted alphabetically by name of person)

**E (max points 5)**. _Nobel prize winners who were born in the same city as their spouses_ (sorted alphabetically by name of person)

**F (max points 5)**. _Politicians that are affiliated with a right-wing party_ (sorted alphabetically by name of person)

Please always show the first 20 entries of the resulting DataFrame and the total count of relevant entries.

## 0.1 Get YAGO data

You will need to download the following datasets that are part of YAGO (see [here](https://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/yago/downloads/) for more information):

* A set of relationships between instances (for example, specifying that Emomali Rahmon is the leader of the Military of Tajikistan). Link: http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoFacts.tsv.7z

* A set of subclass relationships (for example, specifying that *A1086* is *a road in England*, or that *Salmonella Dub* is *a Reggae music group* and also a *New Zealand dub musical group*). Link: http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoTransitiveType.tsv.7z

Please use `wget` to download the data to your compute engine (the files are big!).

Next, you will need extract `tsv` files from the `7z` archives that you have downloaded.
Use the following commands to install `p7zip` on your compute engine and extract the files.
```
sudo apt-get install p7zip-full
7z x yagoTransitiveType.tsv.7z 
7z x yagoFacts.tsv.7z 
```
Please note that this can take a while, in particular as `yagoTransitiveType.tsv` is **18GB** large.

Put the files (`yagoTransitiveType.tsv` and `yagoFacts.tsv`) into the hadoop file system. 
Also, have a look at their first few lines to understand what kind of data they contain.

### Documentation of Initial Steps

First, I open a command prompt window and set up a dataproc cluster `christine-cluster` using the following command:

```
C:\Users\44738>gcloud dataproc clusters create christine-cluster --project seminar-4 --properties=^^^^#^^^^spark:spark.jars.packages=graphframes:graphframes:0.5.0-spark2.1-s_2.11,com.databricks:spark-csv_2.11:1.5.0 --subnet default --zone europe-west2-a --master-machine-type n1-standard-4 --master-boot-disk-size 500 --num-workers 2 --worker-machine-type n1-standard-4 --worker-boot-disk-size 500 --image-version 1.3-deb9  --initialization-actions "gs://dataproc-initialization-actions/jupyter/jupyter.sh","gs://dataproc-initialization-actions/python/pip-install.sh","gs://dataproc-initialization-actions/zookeeper/zookeeper.sh","gs://dataproc-initialization-actions/kafka/kafka.sh" --metadata "PIP_PACKAGES=sklearn nltk pandas graphframes"
Waiting on operation [projects/seminar-4/regions/europe-west2/operations/e1768ca7-97d7-3509-93ae-0569a7524431].
Waiting for cluster creation operation...
WARNING: For PD-Standard without local SSDs, we strongly recommend provisioning 1TB or larger to ensure consistently high I/O performance. See https://cloud.google.com/compute/docs/disks/performance for information on disk I/O performance.
Waiting for cluster creation operation...done.
Created [https://dataproc.googleapis.com/v1/projects/seminar-4/regions/europe-west2/clusters/christine-cluster] Cluster placed in zone [europe-west2-a].
```

Next, I open the hdfs master node using the following command:

```
C:\Users\44738>gcloud beta compute --project "seminar-4" ssh --zone "europe-west2-a" "christine-cluster-m" 
```

In the hdfs master node, I use the `wget` command to download both tsv files:

```
44738@christine-cluster-m:~$ wget http://resources.mpi-inf.mpg.de/yago-naga/yago                                                                             3.1/yagoFacts.tsv.7z
--2020-03-18 21:37:32--  http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoF                                                                             acts.tsv.7z
Resolving resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)... 139.19.206.46
Connecting to resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)|139.19.206.46|                                                                             :80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 240204435 (229M) [application/x-7z-compressed]
Saving to: ‘yagoFacts.tsv.7z’

yagoFacts.tsv.7z    100%[===================>] 229.08M  93.8MB/s    in 2.4s

2020-03-18 21:37:34 (93.8 MB/s) - ‘yagoFacts.tsv.7z’ saved [240204435/240204435]

44738@christine-cluster-m:~$ wget http://resources.mpi-inf.mpg.de/yago-naga/yago                                                                             3.1/yagoTransitiveType.tsv.7z
--2020-03-18 21:37:44--  http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoT                                                                             ransitiveType.tsv.7z
Resolving resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)... 139.19.206.46
Connecting to resources.mpi-inf.mpg.de (resources.mpi-inf.mpg.de)|139.19.206.46|                                                                             :80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1813458594 (1.7G) [application/x-7z-compressed]
Saving to: ‘yagoTransitiveType.tsv.7z’

yagoTransitiveType. 100%[===================>]   1.69G  93.6MB/s    in 18s

2020-03-18 21:38:02 (97.8 MB/s) - ‘yagoTransitiveType.tsv.7z’ saved [1813458594/                                                                             1813458594]

```

Still in the master node, I extract both tsv files according to the directions given above:

```
44738@christine-cluster-m:~$ sudo apt-get install p7zip-full
Reading package lists... Done
Building dependency tree
Reading state information... Done
The following additional packages will be installed:
  p7zip
Suggested packages:
  p7zip-rar
The following NEW packages will be installed:
  p7zip p7zip-full
0 upgraded, 2 newly installed, 0 to remove and 3 not upgraded.
Need to get 1,479 kB of archives.
After this operation, 5,469 kB of additional disk space will be used.
Do you want to continue? [Y/n] y
Get:1 http://deb.debian.org/debian stretch/main amd64 p7zip amd64 16.02+dfsg-3+d                                                                             eb9u1 [364 kB]
Get:2 http://deb.debian.org/debian stretch/main amd64 p7zip-full amd64 16.02+dfs                                                                             g-3+deb9u1 [1,115 kB]
Fetched 1,479 kB in 0s (17.5 MB/s)
Selecting previously unselected package p7zip.
(Reading database ... 142916 files and directories currently installed.)
Preparing to unpack .../p7zip_16.02+dfsg-3+deb9u1_amd64.deb ...
Unpacking p7zip (16.02+dfsg-3+deb9u1) ...
Selecting previously unselected package p7zip-full.
Preparing to unpack .../p7zip-full_16.02+dfsg-3+deb9u1_amd64.deb ...
Unpacking p7zip-full (16.02+dfsg-3+deb9u1) ...
Setting up p7zip (16.02+dfsg-3+deb9u1) ...
Processing triggers for man-db (2.7.6.1-2) ...
Setting up p7zip-full (16.02+dfsg-3+deb9u1) ...

44738@christine-cluster-m:~$ 7z x yagoTransitiveType.tsv.7z

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Int                                                                             el(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 1813458594 bytes (1730 MiB)

Extracting archive: yagoTransitiveType.tsv.7z
--
Path = yagoTransitiveType.tsv.7z
Type = 7z
Physical Size = 1813458594
Headers Size = 148
Method = LZMA:24
Solid = -
Blocks = 1

Everything is Ok

Size:       18390277560
Compressed: 1813458594

44738@christine-cluster-m:~$ 7z x yagoFacts.tsv.7z

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Int                                                                             el(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 240204435 bytes (230 MiB)

Extracting archive: yagoFacts.tsv.7z
--
Path = yagoFacts.tsv.7z
Type = 7z
Physical Size = 240204435
Headers Size = 129
Method = LZMA:24
Solid = -
Blocks = 1

Everything is Ok

Size:       1019693498
Compressed: 240204435
```

Then, I move both files into the hadoop file system:

```
44738@christine-cluster-m:~$ hadoop fs -ls /
Found 3 items
drwx------   - mapred hadoop          0 2020-03-18 21:34 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2020-03-18 21:34 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2020-03-18 21:33 /user

44738@christine-cluster-m:~$ hadoop fs -mkdir /user/root

44738@christine-cluster-m:~$ hadoop fs -put yagoFacts.tsv /user/root/

44738@christine-cluster-m:~$ hadoop fs -put yagoTransitiveType.tsv /user/root/

44738@christine-cluster-m:~$ hadoop fs -ls /user/root
Found 2 items
-rw-r--r--   2 44738 hadoop  1019693498 2020-03-19 18:30 /user/root/yagoFacts.tsv
-rw-r--r--   2 44738 hadoop 18390277560 2020-03-19 18:33 /user/root/yagoTransitiveType.tsv

```

I also took a look at both files to get a sense of what kind of data they contain.

```
44738@christine-cluster-m:~$ head yagoFacts.tsv
        <yagoTheme_yagoFacts>   <hasGloss>      "This file is part of the ontology YAGO3. It is licensed under a Creative-Commons Attribution License by the YAGO team at the Max Planck Institute for Informatics/Germany. See http://yago-knowledge.org for all details. This file was generated on 2017-06-19 T 08:01:29.0836. All facts of YAGO that hold between instances CORE"
<id_rB6isMnplh_H?S_LT?Qo1Fzc!>  <Jesús_Rivera_Sánchez>  <isLeaderOf>    <Pueblo_of_Naranjito>
<id_BOK!FvTDPu_H?S_1NVSTKkFbS>  <Elizabeth_II>  <isLeaderOf>    <Royal_Numismatic_Society>
<id_Uy5EwU3nX1_H?S_otuJrkvKs1>  <Richard_Stallman>      <isLeaderOf>    <Free_Software_Foundation>
<id_A?rIHtKpyX_H?S_Ap3TBzfE6b>  <Keith_Peterson>        <isLeaderOf>    <Cambridge_Bay>
<id_vzhzgmCR5Y_H?S_9xW07qYiaH>  <William_H._Seward_Jr.> <isLeaderOf>    <9th_New_York_Heavy_Artillery_Regiment>
<id_GqUh9jFAN?_H?S_A39fu5FWu4>  <Andranik>      <isLeaderOf>    <Armenian_fedayi>
<id_s60Psk1DHb_H?S_OACCn8W8Kv>  <Ramasamy_Palanisamy>   <isLeaderOf>    <Democratic_Action_Party_(Malaysia)>
<id_pII60Mnz8o_H?S_8mvRWxKXDG>  <Matt_Bevin>    <isLeaderOf>    <Kentucky_Air_National_Guard>
<id_losV58WRWE_H?S_KxmZk2LtbV>  <Leonard_Leo>   <isLeaderOf>    <Federalist_Society>

44738@christine-cluster-m:~$ head yagoTransitiveType.tsv
        <yagoTheme_yagoTransitiveType>  <hasGloss>      "This file is part of the ontology YAGO3. It is licensed under a Creative-Commons Attribution License by the YAGO team at the Max Planck Institute for Informatics/Germany. See http://yago-knowledge.org for all details. This file was generated on 2017-06-06 T 18:48:11.0135. Transitive closure of all rdf:type/rdfs:subClassOf facts TAXONOMY"
<id_IDwmgVA9s0_KCM_rINXattJoD>  <1908_St._Louis_Browns_season>  rdf:type        <wikicat_Baltimore_Orioles_seasons>
<id_IDwmgVA9s0_KCM_w?uK7?WKJH>  <1908_St._Louis_Browns_season>  rdf:type        <wikicat_Major_League_Baseball_teams_seasons>
<id_IDwmgVA9s0_KCM_FyppX?MtG?>  <1908_St._Louis_Browns_season>  rdf:type        <wikicat_St._Louis_Browns_seasons>
<id_IDwmgVA9s0_KCM_2JNSaunrSx>  <1908_St._Louis_Browns_season>  rdf:type        <wordnet_abstraction_100002137>
<id_IDwmgVA9s0_KCM_M8v8FgA!sO>  <1908_St._Louis_Browns_season>  rdf:type        <wordnet_fundamental_quantity_113575869>
<id_IDwmgVA9s0_KCM_G!H3S90mJ0>  <1908_St._Louis_Browns_season>  rdf:type        <wordnet_measure_100033615>
<id_IDwmgVA9s0_KCM_Zlk4NLDCV7>  <1908_St._Louis_Browns_season>  rdf:type        <wordnet_season_115239579>
<id_IDwmgVA9s0_KCM_xxJUNq6YUn>  <1908_St._Louis_Browns_season>  rdf:type        <wordnet_time_period_115113229>
<id_IDwmgVA9s0_KCM_KdEX!y?wiG>  <1908_St._Louis_Browns_season>  rdf:type        owl:Thing
```

Once I have both of my files ready, I proceed to open a PySpark notebook. To do this, I go into Google Cloud Platform and navigate to `Dataproc -> Clusters`, select `christine-cluster`, select `Web Interfaces` and then `Create an SSH tunnel to connect to a web interface`. I use the command given in a second command prompt window:

```
C:\Users\44738>gcloud compute ssh christine-cluster-m --project=seminar-4 --zone=europe-west2-a -- -D 1080 -N
```

In a third command prompt window, I open a new chrome tab by typing the following:

```
C:\Users\44738>cd C:\Program Files (x86)\Google\Chrome\Application

C:\Program Files (x86)\Google\Chrome\Application>chrome.exe --proxy-server="socks5://localhost:1080" --user-data-dir="%Temp%\christine-cluster-m" http://christine-cluster-m:8123
```

I then proceed to open a new PySpark notebook.

## 0.2 Read the data into a Spark DataFrame

Please load the data from `yagoFacts.tsv` into a DataFrame called `df` and `yagoTransitiveType.tsv` into a DataFrame called `df_subclasses`.
Have a look at the beginning of the files to understand the schema.
Once imported, both DataFrames should have columns labelled as `id`, `subject`, `predicate`, `object` and `value`.
In the case of `yagoTransitiveType.tsv`, some of the predicates can be understood as *"is a sublcass of"* or *"is member of the class"*, and the objects can be understood as classes.

In [1]:
# Load libraries
from graphframes import *
from pyspark.sql.types import *

# Set fpath to the location of the data
fpath = "hdfs://christine-cluster-m/user/root/"

# Write a function 'load_tsv' to read in the data
def load_tsv(tsv_name):
    
    # Both DFs have columns id, subject, predicate, object and value
    schema = StructType([
        StructField("id", StringType(), True),    
        StructField("subject", StringType(), True),
        StructField("predicate", StringType(), True),
        StructField("object", StringType(), True),
        StructField("value", StringType(), True)])
    
    # Create DF
    complete_fpath = fpath + tsv_name + '.tsv'
    df = spark.read.csv(complete_fpath, header = 'true', schema = schema, sep = '\t')
    
    return df

# Load the data from 'yagoFacts.tsv' into a DataFrame called 'df'
df = load_tsv('yagoFacts')

# Load the data from 'yagoTransitiveType.tsv' into a DataFrame called 'df_subclasses'
df_subclasses = load_tsv('yagoTransitiveType')

## 0.3 Understand the database schema

Let's look at the schema:

In [2]:
# Have a look at the beginning of both files
df.show(20)
df_subclasses.show(20)

+--------------------+--------------------+------------+--------------------+-----+
|                  id|             subject|   predicate|              object|value|
+--------------------+--------------------+------------+--------------------+-----+
|<id_rB6isMnplh_H?...|<Jesús_Rivera_Sán...|<isLeaderOf>|<Pueblo_of_Naranj...| null|
|<id_BOK!FvTDPu_H?...|      <Elizabeth_II>|<isLeaderOf>|<Royal_Numismatic...| null|
|<id_Uy5EwU3nX1_H?...|  <Richard_Stallman>|<isLeaderOf>|<Free_Software_Fo...| null|
|<id_A?rIHtKpyX_H?...|    <Keith_Peterson>|<isLeaderOf>|     <Cambridge_Bay>| null|
|<id_vzhzgmCR5Y_H?...|<William_H._Sewar...|<isLeaderOf>|<9th_New_York_Hea...| null|
|<id_GqUh9jFAN?_H?...|          <Andranik>|<isLeaderOf>|   <Armenian_fedayi>| null|
|<id_s60Psk1DHb_H?...|<Ramasamy_Palanis...|<isLeaderOf>|<Democratic_Actio...| null|
|<id_pII60Mnz8o_H?...|        <Matt_Bevin>|<isLeaderOf>|<Kentucky_Air_Nat...| null|
|<id_losV58WRWE_H?...|       <Leonard_Leo>|<isLeaderOf>|<Federalist_Society>

The useful information is in columns "subject", "predicate" and "object". "predicate" defines the relation between entities "subject" and "object". For example, for "Albert Einstein was born in Ulm", "Albert Einstein" is the subject, "was born in" is the predicate and "Ulm" is the object.

## 0.4 Simple query example

To get information about where Albert Einstein was born, we load data into Spark using the following query:

In [20]:
born_city_df = df.where("predicate == '<wasBornIn>'")
born_city_df.show(1, False)

+------------------------------+---------------------------------------------+-----------+---------------+-----+
|id                            |subject                                      |predicate  |object         |value|
+------------------------------+---------------------------------------------+-----------+---------------+-----+
|<id_thPX9b1zg!_7fp_kCyegRoKet>|<William_Jones_(Welsh_footballer,_born_1876)>|<wasBornIn>|<Penrhiwceiber>|null |
+------------------------------+---------------------------------------------+-----------+---------------+-----+
only showing top 1 row



In [18]:
born_city_df.where("subject = '<Albert_Einstein>'").show(born_city_df.count(), False)

+------------------------------+-----------------+-----------+------+-----+
|id                            |subject          |predicate  |object|value|
+------------------------------+-----------------+-----------+------+-----+
|<id_sbCVliqDT2_7fp_SjB1FOwfPE>|<Albert_Einstein>|<wasBornIn>|<Ulm> |null |
+------------------------------+-----------------+-----------+------+-----+



You may wonder how one would know whether to use the predicate '&lt;wasBornIn&gt;' or '&lt;was_born_in&gt;' and subject '&lt;Albert_Einstein&gt;' or '&lt;AlbertEinstein&gt;'. For YAGO subjects (and objects), the naming is aligned with Wikipedia. For example, Albert Einstein's wiki is: https://en.wikipedia.org/wiki/Albert_Einstein and you can see it is 'Albert_Einstein'. 

For predicates, you can look at the "property" list from the [yago web interface](https://gate.d5.mpi-inf.mpg.de/webyagospotlx/WebInterface?L01=%3Fx&L0R=%3CwasBornIn%3E&L02=%3Fc&L0T=&L03=&L0L=&L04=&L05=&L11=&L1R=&L12=&L1T=&L13=&L1L=&L14=&L15=&L21=&L2R=&L22=&L2T=&L23=&L2L=&L24=&L25=&L31=&L3R=&L32=&L3T=&L33=&L3L=&L34=&L35=&L41=&L4R=&L42=&L4T=&L43=&L4L=&L44=&L45=). 
Try different queries with this web interface query to understand more how to query YAGO.

## 0.5 Simple motif example (Question A)

In this part of the homework, you are required to use **motif** to find out answer to the 4 questions. Please complete the following example to find out: "Which city was Albert Einstein born in?" using motif queries instead of  SQL queries on the first dataframe (`df`):

In [9]:
# NOTE: I use born_city_df because it is already filtered to predicate = "<WasBornIn>"

# Define vertices and edges
v = born_city_df.select("subject", "id").distinct()
e = born_city_df.withColumnRenamed("subject", "src").withColumnRenamed("object", "dst").select("src", "dst")

# Build the graph
g = GraphFrame(v, e)
print("Graph: "); print(g); print(" ")
print("Vertices: "); g.vertices.show(5, False)
print("Edges: "); g.edges.show(5, False)

Graph: 
GraphFrame(v:[id: string, subject: string], e:[src: string, dst: string])
 
Vertices: 
+---------------------------+------------------------------+
|subject                    |id                            |
+---------------------------+------------------------------+
|<nl/Kars_van_Tarel>        |<id_ygSgtwYWIJ_7fp_lGhmJyFgmN>|
|<James_J._Couzens>         |<id_DCT32lh0dA_7fp_RpE0TDEnyY>|
|<Albert_Russell_(director)>|<id_IjUg1kUfud_7fp_enl8MQVrnA>|
|<Paddy_O'Connor>           |<id_dy1deLP3XL_7fp_Q?oM5KbAHV>|
|<Jackie_Stedall>           |<id_Oz2rAe9QE4_7fp_0mZZ28tJjV>|
+---------------------------+------------------------------+
only showing top 5 rows

Edges: 
+---------------------------------------------+---------------+
|src                                          |dst            |
+---------------------------------------------+---------------+
|<William_Jones_(Welsh_footballer,_born_1876)>|<Penrhiwceiber>|
|<Prince_Konrad_of_Hohenlohe-Schillingsfürst> |<Vienna>       |
|<S

In [10]:
# Write a motif query and filter on Albert Einstein to see where he was born
motifs = g.find("()-[e]->()").filter("e.src = '<Albert_Einstein>'").show(1, False)

+--------------------------+
|e                         |
+--------------------------+
|[<Albert_Einstein>, <Ulm>]|
+--------------------------+



## 0.6 Some useful tips

### Get a subset of YAGO database
YAGO database is large, so we don't try to load the entire database into a dataframe and then query it. If you do this, you will find that you won't even be able to execute `df.take(1)`, as it would take up too much of space (at least on a laptop). Instead, you use Spark SQL commands or `df.where` to get a suitable fraction of the data.

### Try the queries in the YAGO web interface first
It is sometimes tricky to get the right "subject", "predicate" and "object". It is easier if you start from [yago web interface](https://gate.d5.mpi-inf.mpg.de/webyagospotlx/WebInterface?L01=%3Fx&L0R=%3CwasBornIn%3E&L02=%3Fc&L0T=&L03=&L0L=&L04=&L05=&L11=&L1R=&L12=&L1T=&L13=&L1L=&L14=&L15=&L21=&L2R=&L22=&L2T=&L23=&L2L=&L24=&L25=&L31=&L3R=&L32=&L3T=&L33=&L3L=&L34=&L35=&L41=&L4R=&L42=&L4T=&L43=&L4L=&L44=&L45=) rather than directly querying in Pyspark. Once your query works, you can convert your query to Pyspark code. Note that sometimes the web version of object/subject code may be different from what you need to type here. For example, company code is &lt;wordnet_company_108058098&gt; when you do the query here but when you do it via the web interface it is &lt;wordnet company 108058098&gt;. 

### Be patient and don't do this exercise in the last minute
Some trial and error is needed to get the query right and it may take some time get the result for a query. For these reasons, we advise you not to wait to work out this exercise just before the submission deadline. 

### Make sure to get the initialization actions right
For this exercise, you will be using GraphFrames.

## 1. Politicians who are also scientists (Question B)
Find all politicians who are also scientists. Output top 20 of them. How many people are in the dataset who are both scientists and politicians?
Please follow these steps:
* Operate on the subsets of `df_subclasses` where the objects are `'<wordnet_scientist_110560637>` (scientists) and `'<wordnet_politician_110450303>'` (politicians), and where the predicates are `rdf:type`.
* Use graphframes and the right parts of `df_subclasses` to construct a graph whose (directed) edges point from subjects to objects. Hence, its source vertices are subjects and it destination vertices are objects. It may be convenient to use intermediate DataFrames and join all the required dataframes of edges and vertices.
* The subjects will be people and the objects will be classes (e.g., scientists, politicians).
* Use a motif query to find all instances that fulfil the criteria specified in the question.
* It is a good idea to define a function that takes a DataFrame and outputs a set of data frames for vertices and edges.

Please sort the output alphabetically by the person column.

In [3]:
# Create a subset of df_subclasses including only scientists and politicians
poli_sci_df = df_subclasses.where("object == '<wordnet_scientist_110560637>' or object == '<wordnet_politician_110450303>'")

# Additionally, filter on predicate = rdf:type
poli_sci_df = poli_sci_df.where("predicate = 'rdf:type'")

# Display the first 20 rows to make sure this worked
poli_sci_df.show(5)

+--------------------+--------------------+---------+--------------------+-----+
|                  id|             subject|predicate|              object|value|
+--------------------+--------------------+---------+--------------------+-----+
|<id_wGHfubCwBs_KC...|<Jean-Baptiste-Jo...| rdf:type|<wordnet_politici...| null|
|<id_EQgbQobwPR_KC...|       <Reg_Freeson>| rdf:type|<wordnet_politici...| null|
|<id_AZU1dcMWPB_KC...|       <Akbar_Ahmad>| rdf:type|<wordnet_politici...| null|
|<id_OqLYLwYtOx_KC...|<it/Luigi_Di_Paol...| rdf:type|<wordnet_politici...| null|
|<id_X76ScLZ?xM_KC...|         <Larry_Wos>| rdf:type|<wordnet_scientis...| null|
+--------------------+--------------------+---------+--------------------+-----+
only showing top 5 rows



In [6]:
# Define source vertices (subjects)
# NOTE: I need to use dropDuplicates() because for some reason every name shows up 
# two times with two separate IDs and I do not want to do any double counting
source_v = poli_sci_df.select("subject","id").dropDuplicates(["subject"])
print("Source Vertices: "); source_v.show(5, False)

# Define destination vertices (objects)
destination_v = sqlContext.createDataFrame([("<wordnet_politician_110450303>","politician"),
                                            ("<wordnet_scientist_110560637>","scientist")], ["id", "subject"])
print("Destination Vertices: "); destination_v.show(2, False)

# Combine source vertices and destination vertices
v = destination_v.unionAll(source_v)

# Define edges
e = poli_sci_df.withColumnRenamed("subject","src").withColumnRenamed("object","dst").select("src","dst")

g = GraphFrame(v, e)
print("Graph: "); print(g); print(" ")
print("Vertices: "); g.vertices.show(5, False)
print("Edges: "); g.edges.show(5, False)

Source Vertices: 
+-----------------------+------------------------------+
|subject                |id                            |
+-----------------------+------------------------------+
|<A._Richard_Newton>    |<id_d1dFPQY3qb_KCM_shP4LW!R1g>|
|<Abby_Lee_(politician)>|<id_RqACHpnJtB_KCM_aXuIGKiXUR>|
|<Abd_al-Karim_al-Jundi>|<id_aDyZSu5v5f_KCM_aXuIGKiXUR>|
|<Abd_al-Majid_al-Rafei>|<id_Z9leXdc3mN_KCM_aXuIGKiXUR>|
|<Abdul_Ali_Mazari>     |<id_0zWAROacTh_KCM_aXuIGKiXUR>|
+-----------------------+------------------------------+
only showing top 5 rows

Destination Vertices: 
+------------------------------+----------+
|id                            |subject   |
+------------------------------+----------+
|<wordnet_politician_110450303>|politician|
|<wordnet_scientist_110560637> |scientist |
+------------------------------+----------+

Graph: 
GraphFrame(v:[id: string, subject: string], e:[src: string, dst: string])
 
Vertices: 
+------------------------------+-----------------------------

In [10]:
# Motif query
motifs = g.find("(a)-[e]->(b); (a)-[e2]->(c)")

# Filter to include only politicians
poli_motifs = motifs.filter("b.id == '<wordnet_politician_110450303>'")

# Filter and count the number of politicians who are also scientists
poli_sci_motifs = poli_motifs.filter("c.id == '<wordnet_scientist_110560637>'")
print("The total number of politicians that are also scientists is:", poli_sci_motifs.count())

The total number of politicians that are also scientists is: 7182


In [9]:
# Display the top 20 politicians who are also scientists
poli_sci_motifs.select("e.src").orderBy('e.src', ascending=True).show(20, False)

+-------------------------+
|src                      |
+-------------------------+
|<A._C._Cuza>             |
|<A._P._J._Abdul_Kalam>   |
|<Aad_Kosto>              |
|<Aad_Nuis>               |
|<Aaron_Aaronsohn>        |
|<Aaron_Farrugia>         |
|<Ab_Klink>               |
|<Abba_P._Lerner>         |
|<Abbas_Ahmad_Akhoundi>   |
|<Abbie_Hoffman>          |
|<Abbott_Lawrence_Lowell> |
|<Abdallah_Salem_el-Badri>|
|<Abdelbaki_Hermassi>     |
|<Abdellatif_Abid>        |
|<Abdelouahed_Souhail>    |
|<Abdelwahed_Radi>        |
|<Abdesslam_Yassine>      |
|<Abdi_Farah_Shirdon>     |
|<Abdirahman_Duale_Beyle> |
|<Abdiweli_Mohamed_Ali>   |
+-------------------------+
only showing top 20 rows



## 2. Companies whose founders were born in London (Question C)
For companies, use `'<wordnet_company_108058098>'`. 
For *"being founder"*, use `<created>`.

By now, you will understand which DataFrame to use for what. 

Set up a graph and use a motif query to find companies whose founders were born in London.
Please take some time to figure out how a suitable configuration of nodes and edges should look like.  How many such companies are there in our dataset?

Please sort the output alphabetically by the founder column.

In [128]:
# Create a subset of df_subclasses including only companies
company_df = df_subclasses.where("object == '<wordnet_company_108058098>'")

# Create a subset of df including only creators
creator_df = df.where("predicate == '<created>'")

# Display the first 5 rows to make sure this worked; also display born_city_df because we will use it here
company_df.show(5)
creator_df.show(5)
born_city_df.show(5)

+--------------------+--------------------+---------+--------------------+-----+
|                  id|             subject|predicate|              object|value|
+--------------------+--------------------+---------+--------------------+-----+
|<id_mH08dclRGm_KC...|<Ice_Cream_Man_(b...| rdf:type|<wordnet_company_...| null|
|<id_sPDFWJYYVp_KC...|<Bangladesh_Insti...| rdf:type|<wordnet_company_...| null|
|<id_9P3ZvWWaBA_KC...|         <Giacomini>| rdf:type|<wordnet_company_...| null|
|<id_VER4Spac3k_KC...|     <Nfrastructure>| rdf:type|<wordnet_company_...| null|
|<id_xnF4xFL9GN_KC...|<nl/Orania_(aarda...| rdf:type|<wordnet_company_...| null|
+--------------------+--------------------+---------+--------------------+-----+
only showing top 5 rows

+--------------------+------------------+---------+--------------------+-----+
|                  id|           subject|predicate|              object|value|
+--------------------+------------------+---------+--------------------+-----+
|<id_Y2g!

In [131]:
# Define source vertices (founders)
source_v = born_city_df.select("subject","object").withColumnRenamed("subject","id").dropDuplicates(["id"])
print("Source Vertices: "); source_v.show(5, False)

# Define destination vertices (objects)
destination_v = company_df.select("subject","object").withColumnRenamed("subject","id").distinct()
print("Destination Vertices: "); destination_v.show(5, False)

Source Vertices: 
+----------------------------+------------------------+
|id                          |object                  |
+----------------------------+------------------------+
|<2nd_Jebtsundamba_Khutughtu>|<Altanbulag,_Töv>       |
|<A.G._Visser>               |<Fraserburg>            |
|<A._J._Gass>                |<Bellflower,_California>|
|<A._J._Pollock>             |<Hebron,_Connecticut>   |
|<A._K._Sarwate>             |<Amravati>              |
+----------------------------+------------------------+
only showing top 5 rows

Destination Vertices: 
+------------------------------+---------------------------+
|id                            |object                     |
+------------------------------+---------------------------+
|<de/Hanomag_Lohnhärterei>     |<wordnet_company_108058098>|
|<de/Superfilm>                |<wordnet_company_108058098>|
|<S-Bank>                      |<wordnet_company_108058098>|
|<RAP4>                        |<wordnet_company_108058098>|
|<d

In [133]:
# Combine source vertices and destination vertices
v = destination_v.unionAll(source_v)

# Define edges
e = creator_df.withColumnRenamed("subject","src").withColumnRenamed("object","dst").select("src","dst")

# Create the graph
g = GraphFrame(v, e)
print("Graph: "); print(g); print(" ")
print("Vertices: "); g.vertices.show(5, False)
print("Edges: "); g.edges.show(5, False)

Graph: 
GraphFrame(v:[id: string, object: string], e:[src: string, dst: string])
 
Vertices: 
+------------------------------+---------------------------+
|id                            |object                     |
+------------------------------+---------------------------+
|<de/Brauerei_Ritter_St._Georg>|<wordnet_company_108058098>|
|<WestNet_Wireless>            |<wordnet_company_108058098>|
|<New_Ocean_Media>             |<wordnet_company_108058098>|
|<Austen_Morris_Associates>    |<wordnet_company_108058098>|
|<de/Bermuda_Buggy>            |<wordnet_company_108058098>|
+------------------------------+---------------------------+
only showing top 5 rows

Edges: 
+------------------+------------------------+
|src               |dst                     |
+------------------+------------------------+
|<Donald_Moffitt>  |<Second_Genesis_(novel)>|
|<Ludwig_Bemelmans>|<Madeline's_Rescue>     |
|<Hitomi_Shimatani>|<Perseus_(song)>        |
|<Steven_Paul>     |<The_Karate_Dog>        |
|<

In [13]:
# Motif query
# NOTE: apply a filter to eliminate the rare instances where two people are connected by an edge
motifs = g.find("(a)-[e]->(b)").filter("b.object = '<wordnet_company_108058098>'")
motifs.show(10)

+--------------------+--------------------+--------------------+
|                   a|                   e|                   b|
+--------------------+--------------------+--------------------+
|[<Mona_Yamamoto>,...|[<Mona_Yamamoto>,...|[<Asahi_Broadcast...|
|[<Sufjan_Stevens>...|[<Sufjan_Stevens>...|[<Asthmatic_Kitty...|
|[<Charlie_Robinso...|[<Charlie_Robinso...|[<C._H._Robinson>...|
|[<Milt_Gabler>, <...|[<Milt_Gabler>, <...|[<Commodore_Recor...|
|[<Daniel_Cane>, <...|[<Daniel_Cane>, <...|[<CourseInfo>, <w...|
|[<DirecTV>, <word...|[<DirecTV>, <Dire...|[<DirecTV_Now>, <...|
|[<AT&T>, <wordnet...|[<AT&T>, <DirecTV...|[<DirecTV_Now>, <...|
|[<Fifth_Generatio...|[<Fifth_Generatio...|[<FastBack>, <wor...|
|[<Rockwilder>, <Q...|[<Rockwilder>, <F...|[<Fuel_Records>, ...|
|[<Witness_Lee>, <...|[<Witness_Lee>, <...|[<Living_Stream_M...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [20]:
# Filter to include only founders who were born in London
london_motifs = motifs.filter("a.object == '<London>'")

# Count how many company founders were born in London
founder_count = london_motifs.select("e.src").distinct().count()
print("The number of company founders who were born in London is:", founder_count)

# Count how many companies had founders who were born in London
companies_count = london_motifs.select("e.dst").distinct().count()
print("The number of companies whose founders were born in London is:", companies_count)

The number of company founders who were born in London is: 53
The number of companies whose founders were born in London is: 59


In [41]:
# Display a list of the company founders who were born in London
# NOTE: Some names show up more than once because the person founded multiple companies (i.e. Ellen Browning Scripps)
london_motifs.select("e.src","e.dst").orderBy('e.src', ascending=True).show(20,False)

+-------------------------------+-------------------------------------+
|src                            |dst                                  |
+-------------------------------+-------------------------------------+
|<Adam_Hamdy>                   |<Dare_Comics>                        |
|<Alexander_Asseily>            |<Jawbone_(company)>                  |
|<Antony_Jay>                   |<Video_Arts>                         |
|<Aubrey_de_Grey>               |<SENS_Research_Foundation>           |
|<Ben_Horowitz>                 |<Andreessen_Horowitz>                |
|<Bernard_MacMahon_(filmmaker)> |<LO-MAX_Records>                     |
|<Brian_Maxwell>                |<PowerBar>                           |
|<Bruno_Heller>                 |<Primrose_Hill_Productions>          |
|<Charlie_Chaplin>              |<United_Artists>                     |
|<Dan_Joyce>                    |<Kurrupt_Recordings_HARD>            |
|<Daniel_James_(game_developer)>|<Three_Rings_Design>           

## 3. Writers who have won a Nobel Prize in any discipline, including economics (Question D)
Tags for nobel prizes look like these: `'<Nobel_Prize_in_Chemistry>`, `<Nobel_Prize_in_Physics>'`, `<Nobel_Prize>` or `<Nobel_Prize>` etc.
We are also counting this one: `'<Nobel_Memorial_Prize_in_Economic_Sciences>'`.

The tag for writers is `'<wordnet_writer_110794014>'`.

You will need to use `'<hasWonPrize>'` as a predicate.

Please sort the output alphabetically by the person column.

In [122]:
# Create a subset of df_subclasses including only writers
writers_df = df_subclasses.where("object == '<wordnet_writer_110794014>'")

# Create a subset of df including only individuals who have won prizes
prize_winners_df = df.where("predicate == '<hasWonPrize>'")

# Filter prize_winners_df to include only Nobel prizes
prize_winners_df = prize_winners_df.where("object == '<Nobel_Prize>' or \
object == '<Nobel_Peace_Prize>' or object == '<Nobel_Prize_in_Physics>' or \
object == '<Nobel_Prize_in_Literature>' or object == '<Nobel_Prize_in_Chemistry>' or \
object == '<Nobel_Prize_in_Physiology_or_Medicine>' or object == '<Nobel_Memorial_Prize_in_Economic_Sciences>'")

# Display the first 5 rows to make sure this worked
writers_df.show(5)
prize_winners_df.show(5)

+--------------------+--------------------+---------+--------------------+-----+
|                  id|             subject|predicate|              object|value|
+--------------------+--------------------+---------+--------------------+-----+
|<id_Rjw3Ie7SP9_KC...|    <Jessica_Yellin>| rdf:type|<wordnet_writer_1...| null|
|<id_FmLL6QBeue_KC...|<es/Bernardo_Marí...| rdf:type|<wordnet_writer_1...| null|
|<id_2p3oRVZ3Xj_KC...|    <Phyllis_Tickle>| rdf:type|<wordnet_writer_1...| null|
|<id_rn12Pb4Pmh_KC...|    <Roger_McDonald>| rdf:type|<wordnet_writer_1...| null|
|<id_pEyVYsG0Oi_KC...|   <es/Tomás_Lander>| rdf:type|<wordnet_writer_1...| null|
+--------------------+--------------------+---------+--------------------+-----+
only showing top 5 rows

+--------------------+--------------------+-------------+--------------------+-----+
|                  id|             subject|    predicate|              object|value|
+--------------------+--------------------+-------------+-------------------

In [123]:
# Define source vertices (name, prize)
source_v = prize_winners_df.select("subject","object").withColumnRenamed("subject","id").dropDuplicates(["id"])
print("Source Vertices: "); source_v.show(5, False)

# Define destination vertices (writer_id)
destination_v = sqlContext.createDataFrame([("<wordnet_writer_110794014>","writer")], ["id", "object"])
print("Destination Vertices: "); destination_v.show(1, False)


Source Vertices: 
+----------------------------+---------------------------------------+
|id                          |object                                 |
+----------------------------+---------------------------------------+
|<Carl_von_Ossietzky>        |<Nobel_Peace_Prize>                    |
|<Grazia_Deledda>            |<Nobel_Prize_in_Literature>            |
|<Pierre_Curie>              |<Nobel_Prize_in_Physics>               |
|<Richard_R._Schrock>        |<Nobel_Prize_in_Chemistry>             |
|<Stanley_Cohen_(biochemist)>|<Nobel_Prize_in_Physiology_or_Medicine>|
+----------------------------+---------------------------------------+
only showing top 5 rows

Destination Vertices: 
+--------------------------+------+
|id                        |object|
+--------------------------+------+
|<wordnet_writer_110794014>|writer|
+--------------------------+------+



In [124]:
# Combine source vertices and destination vertices
v = destination_v.unionAll(source_v)

# Define edges
e = writers_df.withColumnRenamed("subject","src").withColumnRenamed("object","dst").select("src","dst")

# Build graph
g = GraphFrame(v, e)
print("Graph: "); print(g); print(" ")
print("Vertices: "); g.vertices.show(5, False)
print("Edges: "); g.edges.show(5, False)

Graph: 
GraphFrame(v:[id: string, object: string], e:[src: string, dst: string])
 
Vertices: 
+--------------------------+---------------------------+
|id                        |object                     |
+--------------------------+---------------------------+
|<wordnet_writer_110794014>|writer                     |
|<Carl_von_Ossietzky>      |<Nobel_Peace_Prize>        |
|<Grazia_Deledda>          |<Nobel_Prize_in_Literature>|
|<Pierre_Curie>            |<Nobel_Prize_in_Physics>   |
|<Richard_R._Schrock>      |<Nobel_Prize_in_Chemistry> |
+--------------------------+---------------------------+
only showing top 5 rows

Edges: 
+---------------------------------+--------------------------+
|src                              |dst                       |
+---------------------------------+--------------------------+
|<Jessica_Yellin>                 |<wordnet_writer_110794014>|
|<es/Bernardo_María_de_la_Calzada>|<wordnet_writer_110794014>|
|<Phyllis_Tickle>                 |<wordnet_w

In [125]:
# Motif query
motifs = g.find("(a)-[e]->(b)")
motifs.show(10)

+--------------------+--------------------+--------------------+
|                   a|                   e|                   b|
+--------------------+--------------------+--------------------+
|[<Carl_von_Ossiet...|[<Carl_von_Ossiet...|[<wordnet_writer_...|
|[<Grazia_Deledda>...|[<Grazia_Deledda>...|[<wordnet_writer_...|
|[<W._B._Yeats>, <...|[<W._B._Yeats>, <...|[<wordnet_writer_...|
|[<Tjalling_Koopma...|[<Tjalling_Koopma...|[<wordnet_writer_...|
|[<Henry_Yule>, <N...|[<Henry_Yule>, <w...|[<wordnet_writer_...|
|[<Norman_Borlaug>...|[<Norman_Borlaug>...|[<wordnet_writer_...|
|[<J._Michael_Kost...|[<J._Michael_Kost...|[<wordnet_writer_...|
|[<Élie_Ducommun>,...|[<Élie_Ducommun>,...|[<wordnet_writer_...|
|[<Nadine_Gordimer...|[<Nadine_Gordimer...|[<wordnet_writer_...|
|[<Salvatore_Quasi...|[<Salvatore_Quasi...|[<wordnet_writer_...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [126]:
# Print the number of Nobel Prizes won by writers 
nobel_prizes_count = motifs.count()
print("The number of Nobel Prizes won by writers is:",nobel_prizes_count)

# Check to see if any writers won multiple Nobel Prizes
winners_count = motifs.select("a.id").distinct().count()
print("The number of writers who won Nobel Prizes is:",winners_count)

The number of Nobel Prizes won by writers is: 255
The number of writers who won Nobel Prizes is: 255


In [127]:
# Display a list of writers who have won Nobel Prizes (first 20 only)
new_names = ['Name', 'Nobel Prize']
motifs.select("a.id","a.object").orderBy('a.id').toDF(*new_names).show(20,False)

+------------------------------+-------------------------------------------+
|Name                          |Nobel Prize                                |
+------------------------------+-------------------------------------------+
|<14th_Dalai_Lama>             |<Nobel_Peace_Prize>                        |
|<Adrienne_Clarkson>           |<Nobel_Prize_in_Physics>                   |
|<Al_Gore>                     |<Nobel_Peace_Prize>                        |
|<Albert_Camus>                |<Nobel_Prize_in_Literature>                |
|<Albert_Einstein>             |<Nobel_Prize_in_Physics>                   |
|<Albert_Lutuli>               |<Nobel_Peace_Prize>                        |
|<Albert_Schweitzer>           |<Nobel_Peace_Prize>                        |
|<Aleksandr_Solzhenitsyn>      |<Nobel_Prize_in_Literature>                |
|<Alexander_Prokhorov>         |<Nobel_Prize_in_Physics>                   |
|<Alexei_Alexeyevich_Abrikosov>|<Nobel_Prize_in_Physics>                   |

## 4. Nobel prize winners who were born in the same city as their spouses (Question E)
You may find the predicate `'<isMarriedTo>'` useful to create a Dataframe of all mariages.
Please also show the cities in which the Nobel laureates and their spouses were born.

Please sort the output alphabetically by the person (prize winner) column.

In [107]:
# Create a subset of df including only individuals who have won prizes
spouses_df = df.where("predicate == '<isMarriedTo>'")

# Add a new column to prize_winners_df called 'Nobel Prize'
# NOTE: This dataset has already been filtered to include only nobel prize winners
from pyspark.sql.functions import *
prize_winners_df = prize_winners_df.withColumn("prize",lit("<Nobel_Prize>"))

# Display the dataframes used for this question
spouses_df.show(5)
prize_winners_df.show(5)
born_city_df.show(5)

+--------------------+--------------------+-------------+--------------------+-----+
|                  id|             subject|    predicate|              object|value|
+--------------------+--------------------+-------------+--------------------+-----+
|<id_bv8wQjWcC7_?X...|<Edith_Killgore_K...|<isMarriedTo>|<Claude_Kirkpatrick>| null|
|<id_eqxN5gR1iT_?X...|<Catherine_I,_Lat...|<isMarriedTo>|<Charles,_Count_o...| null|
|<id_ANxEYVaN8C_?X...|<John_Frederick,_...|<isMarriedTo>|<Erdmuthe_of_Bran...| null|
|<id_0UpPzpyhgL_?X...|        <Max_Frisch>|<isMarriedTo>|<Gertrud_Frisch-v...| null|
|<id_h!AfjiGNcx_?X...|          <Chae_Rim>|<isMarriedTo>|    <Lee_Seung-hwan>| null|
+--------------------+--------------------+-------------+--------------------+-----+
only showing top 5 rows

+--------------------+--------------------+-------------+--------------------+-----+-------------+
|                  id|             subject|    predicate|              object|value|        prize|
+-----------

In [108]:
# Define vertices (name, <Nobel_Prize>)
prize_v = prize_winners_df.select("subject","prize").withColumnRenamed("subject","id").withColumnRenamed("prize","object").dropDuplicates(["id"])
print("Vertices (name, <Nobel_Prize>): "); prize_v.show(5, False)

# Define vertices (name, <isMarriedTo>)
marriage_v = spouses_df.select("subject","predicate").withColumnRenamed("subject","id").withColumnRenamed("predicate","object").dropDuplicates(["id"])
print("Vertices (name, <isMarriedTo>): "); marriage_v.show(5, False)

# Define vertices (name, <wasBornIn>)
city_v = born_city_df.select("object","predicate").withColumnRenamed("object","id").withColumnRenamed("predicate","object").dropDuplicates(["id"])
print("Vertices (name, <wasBornIn>): "); city_v.show(5, False)

Vertices (name, <Nobel_Prize>): 
+----------------------------+-------------+
|id                          |object       |
+----------------------------+-------------+
|<Werner_Forssmann>          |<Nobel_Prize>|
|<Carl_von_Ossietzky>        |<Nobel_Prize>|
|<Richard_R._Schrock>        |<Nobel_Prize>|
|<Pierre_Curie>              |<Nobel_Prize>|
|<Stanley_Cohen_(biochemist)>|<Nobel_Prize>|
+----------------------------+-------------+
only showing top 5 rows

Vertices (name, <isMarriedTo>): 
+--------------------------------------+-------------+
|id                                    |object       |
+--------------------------------------+-------------+
|<Acamapichtli>                        |<isMarriedTo>|
|<Adolf_William,_Duke_of_Saxe-Eisenach>|<isMarriedTo>|
|<Agnes_of_Hesse>                      |<isMarriedTo>|
|<Albert,_Duke_of_Prussia>             |<isMarriedTo>|
|<Alexis_Bledel>                       |<isMarriedTo>|
+--------------------------------------+-------------+
only show

In [109]:
# Combine all vertices
v_temp = prize_v.unionAll(marriage_v)
v = city_v.unionAll(v_temp)

# Define marriage edges (name, spouse name)
marriage_e = spouses_df.withColumnRenamed("subject","src").withColumnRenamed("object","dst").select("src","dst")

# Define born city edges (name, city)
city_e = born_city_df.withColumnRenamed("subject","src").withColumnRenamed("object","dst").select("src","dst")

# Combine all edges
e = marriage_e.unionAll(city_e)

# Build graph
g = GraphFrame(v, e)
print("Graph: "); print(g); print(" ")
print("Vertices: "); g.vertices.show(5, False)
print("Edges: "); g.edges.show(5, False)

Graph: 
GraphFrame(v:[id: string, object: string], e:[src: string, dst: string])
 
Vertices: 
+--------------------------+-----------+
|id                        |object     |
+--------------------------+-----------+
|<Aargau>                  |<wasBornIn>|
|<Abancay>                 |<wasBornIn>|
|<Abbeville,_Louisiana>    |<wasBornIn>|
|<Abilene,_Kansas>         |<wasBornIn>|
|<Accoville,_West_Virginia>|<wasBornIn>|
+--------------------------+-----------+
only showing top 5 rows

Edges: 
+-----------------------------------+------------------------------+
|src                                |dst                           |
+-----------------------------------+------------------------------+
|<Edith_Killgore_Kirkpatrick>       |<Claude_Kirkpatrick>          |
|<Catherine_I,_Latin_Empress>       |<Charles,_Count_of_Valois>    |
|<John_Frederick,_Duke_of_Pomerania>|<Erdmuthe_of_Brandenburg>     |
|<Max_Frisch>                       |<Gertrud_Frisch-von_Meyenburg>|
|<Chae_Rim>          

In [115]:
# Motif query
motifs = g.find("(a)-[]->(b); (a)-[]->(c); (b)-[]->(c)")
motifs.show(10)

+--------------------+--------------------+--------------------+
|                   a|                   b|                   c|
+--------------------+--------------------+--------------------+
|[<Otto_(singer)>,...|[<Alessandra_Negr...|[<Brazil>, <wasBo...|
|[<Murilo_Benício>...|[<Alessandra_Negr...|[<Brazil>, <wasBo...|
|[<Paerisades_III>...|[<Kamasarye_Philo...|[<Bosporan_Kingdo...|
|[<Chris_Lazari>, ...|[<Maritsa_Lazari>...|[<Dora,_Cyprus>, ...|
|[<Uzun_Hasan>, <i...|[<Shaykh_Haydar>,...|[<Despina_Khatun>...|
|[<Catherine_Welle...|[<Arthur_Wellesle...|[<Dublin>, <wasBo...|
|[<Said_Gafurov>, ...|[<Darya_Mitina>, ...|[<Moscow>, <wasBo...|
|[<Mariota,_Counte...|[<Donald_of_Islay...|[<Scotland>, <was...|
|[<Nurhayat_Hiçyak...|[<Erhan_Deniz>, <...|[<Turkey>, <wasBo...|
|[<Brian_Grazer>, ...|[<Gigi_Levangie_G...|[<Los_Angeles>, <...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [111]:
# Filter out all extraneous rows
motifs_new = motifs.filter("a.object == '<Nobel_Prize>' and b.object == '<isMarriedTo>' and c.object == '<wasBornIn>'")
motifs_new.show(10,False)

# Print the number of nobel prize winners who were born in the same city as their spouses 
print("The number of nobel prize winners who were born in the same city as their spouses is:",motifs_new.count())

+----------------------------------------+----------------------------------------+---------------------------------+
|a                                       |b                                       |c                                |
+----------------------------------------+----------------------------------------+---------------------------------+
|[<Robert_Hofstadter>, <Nobel_Prize>]    |[<Douglas_Hofstadter>, <isMarriedTo>]   |[<New_York_City>, <wasBornIn>]   |
|[<Carl_Ferdinand_Cori>, <Nobel_Prize>]  |[<Gerty_Cori>, <isMarriedTo>]           |[<Prague>, <wasBornIn>]          |
|[<Al_Gore>, <Nobel_Prize>]              |[<Tipper_Gore>, <isMarriedTo>]          |[<Washington,_D.C.>, <wasBornIn>]|
|[<Frédéric_Joliot-Curie>, <Nobel_Prize>]|[<Irène_Joliot-Curie>, <isMarriedTo>]   |[<Paris>, <wasBornIn>]           |
|[<Irène_Joliot-Curie>, <Nobel_Prize>]   |[<Frédéric_Joliot-Curie>, <isMarriedTo>]|[<Paris>, <wasBornIn>]           |
|[<Jimmy_Carter>, <Nobel_Prize>]         |[<Rosalynn_Car

In [114]:
# Print a list of all nobel prize winners who were born in the same city as their spouses
new_names = ['Prize Winner', 'Spouse', 'Place of Birth']
motifs_new.select("a.id", "b.id", "c.id").orderBy('a.id').toDF(*new_names).show(motifs_new.count(),False)

+-----------------------+-----------------------+------------------+
|Prize Winner           |Spouse                 |Place of Birth    |
+-----------------------+-----------------------+------------------+
|<Al_Gore>              |<Tipper_Gore>          |<Washington,_D.C.>|
|<Carl_Ferdinand_Cori>  |<Gerty_Cori>           |<Prague>          |
|<Fridtjof_Nansen>      |<Eva_Nansen>           |<Oslo>            |
|<Frédéric_Joliot-Curie>|<Irène_Joliot-Curie>   |<Paris>           |
|<Gerty_Cori>           |<Carl_Ferdinand_Cori>  |<Prague>          |
|<Irène_Joliot-Curie>   |<Frédéric_Joliot-Curie>|<Paris>           |
|<Jimmy_Carter>         |<Rosalynn_Carter>      |<Plains,_Georgia> |
|<Robert_Hofstadter>    |<Douglas_Hofstadter>   |<New_York_City>   |
+-----------------------+-----------------------+------------------+



## 5. Politicians that are affiliated with a right-wing party (Question F)

We are looking for all connections of the form `polician -> party`, where party is a right-wing party and politicians are defined above. If one politician is associated with several right wing parties, you may count them several times.

Use `'<isAffiliatedTo>'` to find membership in organisations and `'<wikicat_Right-wing_parties>'` for right-wing parties organisations.

There are multiple ways to do this.

Please sort the output alphabetically by the person (politician) column.

In [116]:
# Create a subset of df_subclasses including only politicians
politician_df = df_subclasses.where("object == '<wordnet_politician_110450303>' and predicate == 'rdf:type'")
politician_df.show(5)

# Create a subset of df_subclasses including only right-wing parties
right_wing_df = df_subclasses.where("object == '<wikicat_Right-wing_parties>' and predicate == 'rdf:type'")
right_wing_df.show(5)

# Create a subset of df which shows affiliations
affiliation_df = df.where("predicate == '<isAffiliatedTo>'")
affiliation_df.show(5)

+--------------------+--------------------+---------+--------------------+-----+
|                  id|             subject|predicate|              object|value|
+--------------------+--------------------+---------+--------------------+-----+
|<id_wGHfubCwBs_KC...|<Jean-Baptiste-Jo...| rdf:type|<wordnet_politici...| null|
|<id_EQgbQobwPR_KC...|       <Reg_Freeson>| rdf:type|<wordnet_politici...| null|
|<id_AZU1dcMWPB_KC...|       <Akbar_Ahmad>| rdf:type|<wordnet_politici...| null|
|<id_OqLYLwYtOx_KC...|<it/Luigi_Di_Paol...| rdf:type|<wordnet_politici...| null|
|<id_9bhVhwkbRF_KC...|<nl/William_Van_R...| rdf:type|<wordnet_politici...| null|
+--------------------+--------------------+---------+--------------------+-----+
only showing top 5 rows

+--------------------+--------------------+---------+--------------------+-----+
|                  id|             subject|predicate|              object|value|
+--------------------+--------------------+---------+--------------------+-----+
|<i

In [117]:
# Define source vertices (name, affiliation)
source_v = politician_df.select("subject","object").withColumnRenamed("subject","id").dropDuplicates(["id"])
print("Source Vertices: "); source_v.show(5, False)

# Define destination vertices (politician)
destination_v = right_wing_df.select("subject","object").withColumnRenamed("subject","id").dropDuplicates(["id"])
print("Destination Vertices: "); destination_v.show(5, False)

Source Vertices: 
+------------------------+------------------------------+
|id                      |object                        |
+------------------------+------------------------------+
|<Abby_Lee_(politician)> |<wordnet_politician_110450303>|
|<Abd_al-Karim_al-Jundi> |<wordnet_politician_110450303>|
|<Abd_al-Majid_al-Rafei> |<wordnet_politician_110450303>|
|<Abdul_Ali_Mazari>      |<wordnet_politician_110450303>|
|<Abdul_Salam_al-Buseiri>|<wordnet_politician_110450303>|
+------------------------+------------------------------+
only showing top 5 rows

Destination Vertices: 
+----------------------------------+----------------------------+
|id                                |object                      |
+----------------------------------+----------------------------+
|<Alliance_of_Vojvodina_Hungarians>|<wikicat_Right-wing_parties>|
|<Botswana_Democratic_Party>       |<wikicat_Right-wing_parties>|
|<National_Democracy>              |<wikicat_Right-wing_parties>|
|<National_Repub

In [118]:
# Combine source vertices and destination vertices
v = destination_v.unionAll(source_v)

# Define edges
e = affiliation_df.withColumnRenamed("subject","src").withColumnRenamed("object","dst").select("src","dst")

# Build graph
g = GraphFrame(v, e)
print("Graph: "); print(g); print(" ")
print("Vertices: "); g.vertices.show(5, False)
print("Edges: "); g.edges.show(5, False)

Graph: 
GraphFrame(v:[id: string, object: string], e:[src: string, dst: string])
 
Vertices: 
+----------------------------------+----------------------------+
|id                                |object                      |
+----------------------------------+----------------------------+
|<Alliance_of_Vojvodina_Hungarians>|<wikicat_Right-wing_parties>|
|<Botswana_Democratic_Party>       |<wikicat_Right-wing_parties>|
|<National_Democracy>              |<wikicat_Right-wing_parties>|
|<National_Republican_Movement>    |<wikicat_Right-wing_parties>|
|<New_Renaissance_Party>           |<wikicat_Right-wing_parties>|
+----------------------------------+----------------------------+
only showing top 5 rows

Edges: 
+--------------------+------------------------------------+
|src                 |dst                                 |
+--------------------+------------------------------------+
|<Willie_Logie>      |<Alloa_Athletic_F.C.>               |
|<Nico_Zaldana>      |<Feyenoord_Academ

In [119]:
# Motif query
motifs = g.find("(a)-[e]->(b)")
motifs.show(10)

+--------------------+--------------------+--------------------+
|                   a|                   e|                   b|
+--------------------+--------------------+--------------------+
|[<István_Pásztor_...|[<István_Pásztor_...|[<Alliance_of_Voj...|
|[<József_Kasza>, ...|[<József_Kasza>, ...|[<Alliance_of_Voj...|
|[<de/Andor_Deli>,...|[<de/Andor_Deli>,...|[<Alliance_of_Voj...|
|[<Sándor_Egeresi>...|[<Sándor_Egeresi>...|[<Alliance_of_Voj...|
|[<Festus_Mogae>, ...|[<Festus_Mogae>, ...|[<Botswana_Democr...|
|[<Ian_Khama>, <wo...|[<Ian_Khama>, <Bo...|[<Botswana_Democr...|
|[<Nonofo_Molefhi>...|[<Nonofo_Molefhi>...|[<Botswana_Democr...|
|[<Kitso_Mokaila>,...|[<Kitso_Mokaila>,...|[<Botswana_Democr...|
|[<Pelonomi_Venson...|[<Pelonomi_Venson...|[<Botswana_Democr...|
|[<Ponatshego_Kedi...|[<Ponatshego_Kedi...|[<Botswana_Democr...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [120]:
# Make sure there are no extraneous rows
politician_motifs = motifs.filter("a.object == '<wordnet_politician_110450303>' and b.object == '<wikicat_Right-wing_parties>'")

# Display the number of politicians associated with a right-wing party
# NOTE: This number includes repeat counts for a single politician with more than one right-wing party affiliation
print("The number of politicians associated with a right-wing party (with duplicates) is:", politician_motifs.count())

# Display the number of politicians associated with a right-wing party
# NOTE: This number includes a single count for each politician (regardless if they have multiple right-wing party affiliations)
politicians_count = politician_motifs.select("e.src").distinct().count()
print("The number of politicians associated with a right-wing party (no duplicates) is:", politicians_count)

The number of politicians associated with a right-wing party (with duplicates) is: 32243
The number of politicians associated with a right-wing party (no duplicates) is: 29928


In [121]:
# Display the first 20 politicians associated with a right-wing party (incl. duplicates)
new_names = ['Name', 'Political Party Affiliation']
politician_motifs.select("e.src","e.dst").orderBy('e.src', 'e.dst').toDF(*new_names).show(20,False)

+-------------------------------+--------------------------------------+
|Name                           |Political Party Affiliation           |
+-------------------------------+--------------------------------------+
|<A.N.M._Ehsanul_Hoque_Milan>   |<Bangladesh_Nationalist_Party>        |
|<A._A._Wijethunga>             |<United_National_Party>               |
|<A._B._Colton>                 |<Republican_Party_(United_States)>    |
|<A._C._Clemons>                |<Republican_Party_(United_States)>    |
|<A._C._Gibbs>                  |<Republican_Party_(United_States)>    |
|<A._C._Hamlin>                 |<Republican_Party_(United_States)>    |
|<A._Clifford_Jones>            |<Republican_Party_(United_States)>    |
|<A._Dean_Jeffs>                |<Republican_Party_(United_States)>    |
|<A._F._M._Ahsanuddin_Chowdhury>|<Jatiya_Party_(Ershad)>               |
|<A._G._Crowe>                  |<Republican_Party_(United_States)>    |
|<A._Homer_Byington>            |<National_Union_Pa